In [1]:
import pandas as pd
import warnings
from surprise import accuracy, Dataset, KNNWithMeans, Reader
from surprise.model_selection import train_test_split

warnings.filterwarnings('ignore')

# Identificação e nome dos filmes.

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv', sep=',', usecols=["movieId","title"])

# Avaliação dos filmes, pelos usuarios.

In [3]:
ratings_moviesIds = pd.read_csv('ml-latest-small/ratings.csv', sep=',', usecols=["userId","movieId","rating"])

# Filmes ainda não assistido por cada usuario "userId".

In [4]:
unseen_moviesId = dict()

for userId in set(ratings_moviesIds["userId"].values):
    unseen_moviesId[userId] = set(movies["movieId"].values).difference(set(ratings_moviesIds.set_index("userId").ix[userId]["movieId"].values))

# Definição da faixa de notas para os filmes, e leitura das avaliações dos usuarios.

In [5]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_moviesIds[["userId","movieId","rating"]], reader)

# Tratamento dos dados e treimanento do algoritmo de recomendação.

In [6]:
trainset,testset = train_test_split(data,test_size =.2)

algo = KNNWithMeans(k=4, sim_options={'name': 'cosine', 'user_based': True})

algo.fit(trainset)
predictions = algo.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


# Quando se passa o arquivo com os usuarios, a função "train_test_split", na celula anterior, converte os userId bruto do arquivo de entrada, para um ID interno.

# A função abaixo converte o valor bruto do userId que se deseja pesquisar, para o seu ID interno correspondente, e ao receber os ID interno dos vizinhos, converte eles para userId bruto do arquivo, para então retornar uma lista contendo os vizinhos.

In [7]:
def three_best_neighbors(userId):
    return [algo.trainset.to_raw_uid(uid) for uid in algo.get_neighbors(iid=algo.trainset.to_inner_uid(userId), k=3)]

# Recebe o userId do qual se deseja recomendações de filmes, e retorna os 5 filmes mais bem ranqueados pelo algoritmo de recomendação.

In [8]:
def five_best_recommendations(userId): 
    recommendations = dict()
    
    for movieId in unseen_moviesId[userId]: 
        recommendations[movieId]= algo.predict(uid=userId,iid=movieId).est
    
    return [movies.set_index("movieId").ix[movieId[0]][0] for movieId in sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[0:5]]

# Demonstração de funcionamento do codigo

In [9]:
userId = input("Entre com o Id do usuario: ")

print("")
print("As top-5 recomendações:")
for movie in five_best_recommendations(int(userId)):
    print ("\t" + movie),

print("")
print("Os três melhores vizinhos: "+ str(three_best_neighbors(int(userId))))

Entre com o Id do usuario: 671

As top-5 recomendações:
	Lamerica (1994)
	Confessional, The (Confessionnal, Le) (1995)
	White Balloon, The (Badkonake sefid) (1995)
	Aspen (1991)
	Timecrimes (Cronocrímenes, Los) (2007)

Os três melhores vizinhos: [308, 490, 179]
